In [22]:
!pip install langchain google-search-results langchain_core google-generativeai langchain_google_genai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [4]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or getpass("enter langsmith key")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "intro-to-agents"

In [30]:
from langchain_core.tools import tool

@tool
def add(x: float,y: float) -> float:
  """Add 'x' with 'y'."""
  return x+y
@tool
def subtract(x: float,y: float) -> float:
  """subtract 'y' from 'x'."""
  return x-y
@tool
def multiply(x: float,y: float) -> float:
  """multiply 'x' with 'y'."""
  return x*y
@tool
def exponentiate(x: float,y: float) -> float:
  """Raise 'x' to the power of 'y'."""
  return x**y

In [31]:
print(f"add.name={add.name}\nadd.description={add.description}")

add.name=add
add.description=Add 'x' with 'y'.


In [32]:
add.args_schema.model_json_schema()

{'description': "Add 'x' with 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

####The llm output will be string and we have to unpack to json and then passed to tool as **kwargs

In [33]:
import json

llm_output_str = '{"x":5,"y":10}'
llm_output_dict = json.loads(llm_output_str)
llm_output_dict

{'x': 5, 'y': 10}

In [34]:
add.func(**llm_output_dict)

15

Till here we are done with tools and now lets create an agent

##Creating an Agent

In [35]:
from langchain_core.prompts import ChatPromptTemplate ,MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful ai assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]

)
prompt_template

ChatPromptTemplate(input_variables=['chat_history', 'input'], optional_variables=['agent_scratchpad'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [42]:
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(api_key=getpass("enter gemini api key"),model="gemini-2.0-flash",temperature=0.0)


enter gemini api key··········


well be working with old version of agent for now so gonna use depriciated memory

In [50]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [51]:
from langchain.agents import create_tool_calling_agent

tools = [add,subtract,divide,multiply,exponentiate]
agent = create_tool_calling_agent(
    llm=llm,tools=tools,prompt=prompt_template
)


In [52]:
agent.invoke({
    "input":"what is 80.9 multiplied by 10",
    "chat_history":memory.chat_memory.messages,
    "intermediate_steps":[] #agent will append its internal steps
})

[ToolAgentAction(tool='multiply', tool_input={'y': 80.9, 'x': 10.0}, log="\nInvoking: `multiply` with `{'y': 80.9, 'x': 10.0}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 80.9, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a0ddaf4a-d46c-4799-937c-8468af6caebd-0', tool_calls=[{'name': 'multiply', 'args': {'y': 80.9, 'x': 10.0}, 'id': '43b01975-2a8e-4691-9d50-35102066791c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 5, 'total_tokens': 109, 'input_token_details': {'cache_read': 0}})], tool_call_id='43b01975-2a8e-4691-9d50-35102066791c')]

###Here we can only see which function to call and what are the inputs to that function , but in order to execute the function we have to use agent executor

In [53]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [54]:
agent_executor.invoke({
    "input":"My name is shanks",
    "chat_history":memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...
Okay, nice to meet you Shanks!


> Finished chain.


{'input': 'My name is shanks',
 'chat_history': [HumanMessage(content='My name is shanks', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay, nice to meet you Shanks!\n', additional_kwargs={}, response_metadata={})],
 'output': 'Okay, nice to meet you Shanks!\n'}

In [55]:
agent_executor.invoke({
    "input":"what is 80.9 multiplied by 10",
    "chat_history":memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 10.0, 'x': 80.9}`


809.080.9 multiplied by 10 is 809.0


> Finished chain.


{'input': 'what is 80.9 multiplied by 10',
 'chat_history': [HumanMessage(content='My name is shanks', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay, nice to meet you Shanks!\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is 80.9 multiplied by 10', additional_kwargs={}, response_metadata={}),
  AIMessage(content='80.9 multiplied by 10 is 809.0\n', additional_kwargs={}, response_metadata={})],
 'output': '80.9 multiplied by 10 is 809.0\n'}

In [56]:
agent_executor.invoke({
    "input":"Given two numbers, 𝑥 = 5.0 and y=2.0, compute the value of the following expression: ( 𝑥 + 𝑦 )^2 − 𝑥 × 𝑦",
    "chat_history":memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'y': 2.0, 'x': 5.0}`


7.0
Invoking: `add` with `{'y': 2.0, 'x': 5.0}`


7.0
Invoking: `exponentiate` with `{'y': 2.0, 'x': 7.0}`


49.0
Invoking: `multiply` with `{'y': 2.0, 'x': 5.0}`


10.0
Invoking: `add` with `{'y': 2.0, 'x': 5.0}`


7.0
Invoking: `exponentiate` with `{'y': 2.0, 'x': 7.0}`


49.0
Invoking: `multiply` with `{'y': 2.0, 'x': 5.0}`


10.0
Invoking: `subtract` with `{'y': 10.0, 'x': 49.0}`


39.0The value of the expression (𝑥 + 𝑦)^2 − 𝑥 × 𝑦, where 𝑥 = 5.0 and y=2.0, is 39.0.


> Finished chain.


{'input': 'Given two numbers, 𝑥 = 5.0 and y=2.0, compute the value of the following expression: ( 𝑥 + 𝑦 )^2 − 𝑥 × 𝑦',
 'chat_history': [HumanMessage(content='My name is shanks', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay, nice to meet you Shanks!\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is 80.9 multiplied by 10', additional_kwargs={}, response_metadata={}),
  AIMessage(content='80.9 multiplied by 10 is 809.0\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Given two numbers, 𝑥 = 5.0 and y=2.0, compute the value of the following expression: ( 𝑥 + 𝑦 )^2 − 𝑥 × 𝑦', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The value of the expression (𝑥 + 𝑦)^2 − 𝑥 × 𝑦, where 𝑥 = 5.0 and y=2.0, is 39.0.\n', additional_kwargs={}, response_metadata={})],
 'output': 'The value of the expression (𝑥 + 𝑦)^2 − 𝑥 × 𝑦, where 𝑥 = 5.0 and y=2.0, is 39.0.\n'}

In [57]:
memory.chat_memory.messages


[HumanMessage(content='My name is shanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Okay, nice to meet you Shanks!\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 80.9 multiplied by 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='80.9 multiplied by 10 is 809.0\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Given two numbers, 𝑥 = 5.0 and y=2.0, compute the value of the following expression: ( 𝑥 + 𝑦 )^2 − 𝑥 × 𝑦', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The value of the expression (𝑥 + 𝑦)^2 − 𝑥 × 𝑦, where 𝑥 = 5.0 and y=2.0, is 39.0.\n', additional_kwargs={}, response_metadata={})]

In [58]:
agent_executor.invoke({
    "input":"What is my name",
    "chat_history":memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...
Your name is Shanks.


> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='My name is shanks', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay, nice to meet you Shanks!\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is 80.9 multiplied by 10', additional_kwargs={}, response_metadata={}),
  AIMessage(content='80.9 multiplied by 10 is 809.0\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Given two numbers, 𝑥 = 5.0 and y=2.0, compute the value of the following expression: ( 𝑥 + 𝑦 )^2 − 𝑥 × 𝑦', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The value of the expression (𝑥 + 𝑦)^2 − 𝑥 × 𝑦, where 𝑥 = 5.0 and y=2.0, is 39.0.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Shanks.\n', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Shanks.\n'}